In [3]:
import os
import numpy as np

# Define path to .npy files
npy_folder = "/Users/User/Documents/Project_JEB/JEB_Data/SimJEB_Occupancy_Grids"  # Change to your actual path

# Find max dimensions
max_shape = [0, 0, 0]

for file in os.listdir(npy_folder):
    if file.endswith(".npy"):
        data = np.load(os.path.join(npy_folder, file))
        shape = data.shape
        max_shape = [max(max_shape[i], shape[i]) for i in range(len(shape))]

print("Target Shape:", max_shape)


Target Shape: [88, 201, 122]


In [4]:
def resize_grid(grid, target_shape):
    """Resize the occupancy grid by padding or cropping."""
    resized_grid = np.zeros(target_shape, dtype=grid.dtype)

    min_shape = [min(grid.shape[i], target_shape[i]) for i in range(len(target_shape))]

    # Copy data into the resized grid
    resized_grid[:min_shape[0], :min_shape[1], :min_shape[2]] = grid[:min_shape[0], :min_shape[1], :min_shape[2]]
    return resized_grid

# Save resized grids
resized_folder = "/Users/User/Documents/Project_JEB/JEB_Data/resized_grids"
os.makedirs(resized_folder, exist_ok=True)

for file in os.listdir(npy_folder):
    if file.endswith(".npy"):
        grid = np.load(os.path.join(npy_folder, file))
        resized_grid = resize_grid(grid, max_shape)  # Resize
        np.save(os.path.join(resized_folder, file), resized_grid)

print("All grids resized and saved to:", resized_folder)


All grids resized and saved to: /Users/User/Documents/Project_JEB/JEB_Data/resized_grids


In [5]:
# Load resized grids
X = []
file_names = []

for file in os.listdir(resized_folder):
    if file.endswith(".npy"):
        data = np.load(os.path.join(resized_folder, file)).flatten()  # Flatten to 1D
        X.append(data)
        file_names.append(file)

X = np.array(X)

In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the Autoencoder
input_dim = X.shape[1]  # Flattened input size

encoder = models.Sequential([
    layers.InputLayer(input_shape=(input_dim,)),
    layers.Dense(256, activation="relu"),
    layers.Dense(128, activation="relu"),
    layers.Dense(50, activation="relu")  # Reduced dimension
])

decoder = models.Sequential([
    layers.InputLayer(input_shape=(50,)),
    layers.Dense(128, activation="relu"),
    layers.Dense(256, activation="relu"),
    layers.Dense(input_dim, activation="sigmoid")  # Output size same as input
])

autoencoder = models.Sequential([encoder, decoder])
autoencoder.compile(optimizer="adam", loss="mse")

# Train Autoencoder
autoencoder.fit(X, X, epochs=50, batch_size=16, validation_split=0.2)

# Get reduced representations
X_reduced = encoder.predict(X)

# Save reduced .npy files
reduced_folder = "/Users/User/Documents/Project_JEB/JEB_Data/compressed_grids"
os.makedirs(reduced_folder, exist_ok=True)

# Save reduced occupancy grids
for i, file in enumerate(file_names):
    np.save(os.path.join(reduced_folder, file), X_reduced[i])

print("Autoencoder-based Reduction Done! Saved in:", reduced_folder)


/Users/User/Documents/Project_JEB/.venv/lib/python3.12/site-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
